##Read in data for the first time

##Bring in libraries and credentials

In [1]:
import datetime
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *
import time 
Tokens = tls.get_credentials_file()['stream_ids']

In [2]:
AllData=pd.read_csv('K:\Rainshelter\Fodderbeet 2016-2017\LoggedData\RainShelterAlpha_CS650.dat', #specify file path for data to read in
                         parse_dates=True, #tell the function to parse date columns to datetime formats
                         dayfirst=True, #tell the function that the day is before the year in the data i.e format='%d/%m/%Y %H:%M'
                         skiprows = [0,2,3], #leave out rows 1, 3 and 4 which have redundant information
                         index_col = 0, #Use the first column, which is Date, as an index
                         na_values = 'NAN')

##Apply indexing for the first time

In [4]:
#Bring in index data
AllDataIndex=pd.read_csv('D:\GitHubRepos\RainShelterWheat\IndexFiles\SoilWaterAndTempIndex.csv',
                         index_col = 0)
#Apply indexes to data
AllDataTransposed = AllData.transpose()
AllDataIndexed = pd.concat([AllDataIndex,AllDataTransposed], axis=1)
AllDataIndexed.index.name='ColumnHeader'
AllDataIndexed.set_index(['Measurement','Depth','Nitrogen','Irrigation','Plot','Sensor', 'MUX', 'Port','Units','Summary','Block','Treatment'], 
                        append=False, inplace=True)
AllDataIndexed.sort_index(inplace=True)
FieldData=AllDataIndexed.transpose()
FieldData.index = FieldData.index.to_datetime()  ## for some reason the concat function changes the data type on the date indes so need to change it back

#Record the last row number read in
LastRow = FieldData.index.size
np.save('LastRow.npy',LastRow)
FieldData.to_pickle('.\FieldData.pkl')

##Function to read in fresh data and update status files

In [9]:
#Bring in date recorded since last update
def UpdataDataFrame():
    #Bring in previous status files
    LastRead = np.load('.\LastRow.npy')
    FieldData = pd.read_pickle('.\FieldData.pkl')
    #Bring in index data
    AllDataIndex=pd.read_csv('D:\GitHubRepos\RainShelterWheat\IndexFiles\SoilWaterAndTempIndex.csv',
                         index_col = 0)
    
    #Bring in fresh data
    StartRead = LastRead + 4
    Skips = [0,2,3] + list(range(4,StartRead))
    FreshData=pd.read_csv('K:\Rainshelter\Fodderbeet 2016-2017\LoggedData\RainShelterAlpha_CS650.dat', #specify file path for data to read in
                             parse_dates=True, #tell the function to parse date columns to datetime formats
                             dayfirst=True, #tell the function that the day is before the year in the data i.e format='%d/%m/%Y %H:%M'
                             skiprows = Skips, #rows that have already be read
                             index_col = 0, #Use the first column, which is Date, as an index
                             na_values = 'NAN')

    #Apply indexes to fresh data
    FreshDataTransposed = FreshData.transpose()
    FreshDataIndexed = pd.concat([AllDataIndex,FreshDataTransposed], axis=1)
    FreshDataIndexed.index.name='ColumnHeader'
    FreshDataIndexed.set_index(['Measurement','Depth','Nitrogen','Irrigation','Plot','Sensor', 'MUX', 'Port','Units','Summary','Block','Treatment'], 
                            append=False, inplace=True)
    FreshDataIndexed.sort_index(inplace=True)
    NewData=FreshDataIndexed.transpose()
    
    #Join fresh data onto existing data
    FieldData = pd.concat([FieldData,NewData])
    FieldData.sort_index(inplace = True)
    
    #Update status files
    LastRow = FieldData.index.size
    np.save('LastRow.npy',LastRow)
    FieldData.to_pickle('.\FieldData.pkl')
    return FieldData

##Create object for each treament

In [5]:
#Treatment class constructor
class Treatment(object):
    def __init__(self, Number, Irrigation, Nitrogen):
        self.Number=Number
        self.Irrigation=Irrigation
        self.Nitrogen=Nitrogen
        self.streamID=Tokens[number]
        self.Label= Irrigation + ' ' + Nitrogen + 'N'
        self.Trace = Scatter(x=[],y=[], name = self.Label,
                stream = Stream(token=self.streamID)
                )
        self.Stream = py.Stream(self.streamID)

#Creat class with treatment data
Treatments = []
Colors = ['red','blue','green']
NRates = ['0','50','250']
IRates = ['Dryland','Irrigated']
Lines = ['solid','dash']
number = 0
LinePos = 0
for I in IRates:
    ColorPos = 0
    for N in NRates:
        Treatments.append(Treatment(number, I, N))
        Treatments[number].Trace.line = Line(color = Colors[ColorPos], dash = Lines[LinePos])
        ColorPos +=1
        number += 1
    LinePos+=1
    

##Build base Plotly Graph

In [6]:
GraphData = Data([Treatments[0].Trace,
                  Treatments[1].Trace,
                  Treatments[2].Trace,
                  Treatments[3].Trace,
                  Treatments[4].Trace,
                  Treatments[5].Trace
                 ])

layout = Layout(title='Wheat Soil Water Deficits',
                xaxis=XAxis(title='Date'),
                yaxis=YAxis(title='Soil Water Deficit (mm)'))

Fig = Figure(data=GraphData, layout=layout)

unique_url = py.plot(Fig, filename = 'RainShelterWheat')

In [ ]:
for T in Treatments:
    print(T.streamID)

##Function to Updata data frame and traces and stream to Plotly

In [22]:
def Update():
    #Update Data Frame with data that has been logged since last update
    FieldData = UpdataDataFrame()
    #Calculate treatment means omitting data prior to 2014-11-05 08:00:00 to avoid NaN values
    DataMeans =  FieldData.ix['2016-11-16':].groupby(level=['Measurement','Depth','Irrigation', 'Nitrogen'],axis=1).mean()
    DataMeans =  DataMeans.dropna(axis=1) #For some reason it keeps non valid combinations in so need to extract with this function
    DataMeans = DataMeans.resample('4H').last()
    
    #Calculate the water content of the soil profile by multiplying the volumetric water content by each layers
    #depth and summing.  The 0-15 layers are divided by 2 to average the two readings
    ProfileWater = DataMeans.VolumetricWaterContent.ix[:,'D1I'] * 150/2 + \
                   DataMeans.VolumetricWaterContent.ix[:,'D1B'] * 150/2 + \
                   DataMeans.VolumetricWaterContent.ix[:,'D2'] * 150 + \
                   DataMeans.VolumetricWaterContent.ix[:,'D3'] * 300 + \
                   DataMeans.VolumetricWaterContent.ix[:,'D4'] * 300 + \
                   DataMeans.VolumetricWaterContent.ix[:,'D5'] * 300 + \
                   DataMeans.VolumetricWaterContent.ix[:,'D6'] * 300 + \
                   DataMeans.VolumetricWaterContent.ix[:,'D7'] * 300

    FieldCapacity = ProfileWater.resample('D').max()
    FieldCapacity = FieldCapacity.ix['2016-11-16'] +30 # I would have though this would return a data frame with a single row but instead it returns a series with a multiindex in columns
    SoilWaterDeficit = -(FieldCapacity - ProfileWater)    # This calculation only works because field capacity is a multi index series

    X = SoilWaterDeficit.index
    for Treatment in Treatments:
        Treatment.Stream.open()
        Treatment.Stream.write({'x':X,'y':SoilWaterDeficit.ix[:,(Treatment.Irrigation,Treatment.Nitrogen)]})
        Treatment.Stream.close()

In [21]:
FieldData

Measurement         ElectricalConductivity                                    \
Depth                                  D1B                                     
Nitrogen                                 0                                     
Irrigation                         Dryland                         Irrigated   
Plot                                   105     108     116     121       103   
Sensor                                  34      58     122     162        18   
MUX                                      2       2       4       6         1   
Port                                     2      26      26       2        18   
Units                                 dS/m    dS/m    dS/m    dS/m      dS/m   
Summary                               Samp    Samp    Samp    Samp      Samp   
Block                                    1       2       3       4         1   
Treatment                                4       6       2       4         2   
2016-11-16 06:30:00                 0.0801  0.0703  0.0590  0.0788    0.0605   
2016-11-16 06:45:00                 0.0800  0.0703  0.0588  0.0791    0.0604   
2016-11-16 07:00:00                 0.0802  0.0702  0.0590  0.0787    0.0605   
2016-11-16 07:15:00                 0.0801  0.0702  0.0589  0.0790    0.0604   
2016-11-16 07:30:00                 0.0799  0.0702  0.0589  0.0785    0.0604   
2016-11-16 07:45:00                 0.0802  0.0704  0.0590  0.0786    0.0603   
2016-11-16 08:00:00                 0.0803  0.0704  0.0588  0.0788    0.0604   
2016-11-16 08:15:00                 0.0800  0.0705  0.0589  0.0786    0.0605   
2016-11-16 08:30:00                 0.0803  0.0705  0.0591  0.0790    0.0605   
2016-11-16 08:45:00                 0.0805  0.0706  0.0591  0.0791    0.0606   
2016-11-16 09:00:00                 0.0806  0.0709  0.0594  0.0789    0.0608   
2016-11-16 09:15:00                 0.0807  0.0711  0.0595  0.0796    0.0610   
2016-11-16 09:30:00                 0.0812  0.0710  0.0595  0.0795    0.0611   
2016-11-16 09:45:00                 0.0812  0.0709  0.0596  0.0797    0.0613   
2016-11-16 10:00:00                 0.0816  0.0713  0.0598  0.0796    0.0612   
2016-11-16 10:15:00                 0.0815  0.0712  0.0599  0.0800    0.0613   
2016-11-16 10:30:00                 0.0815  0.0716  0.0601  0.0799    0.0616   
2016-11-16 10:45:00                 0.0818  0.0718  0.0604  0.0806    0.0618   
2016-11-16 11:00:00                 0.0826  0.0723  0.0607  0.0807    0.0622   
2016-11-16 11:15:00                 0.0827  0.0725  0.0609  0.0812    0.0626   
2016-11-16 11:30:00                 0.0831  0.0728  0.0611  0.0816    0.0628   
2016-11-16 11:45:00                 0.0834  0.0731  0.0612  0.0818    0.0631   
2016-11-16 12:00:00                 0.0840  0.0737  0.0618  0.0823    0.0633   
2016-11-16 12:15:00                 0.0847  0.0741  0.0621  0.0829    0.0639   
2016-11-16 12:30:00                 0.0850  0.0735  0.0624  0.0835    0.0644   
2016-11-16 12:45:00                 0.0879  0.0751  0.0848  0.0981    0.0830   
2016-11-16 13:00:00                 0.0883  0.0768  0.0837  0.0994    0.0804   
2016-11-16 13:15:00                 0.0894  0.0782  0.0819  0.0992    0.0788   
2016-11-16 13:30:00                 0.0898  0.0789  0.0808  0.0991    0.0775   
2016-11-16 13:45:00                 0.0899  0.0791  0.0788  0.0977    0.0768   
...                                    ...     ...     ...     ...       ...   
2016-11-16 23:30:00                 0.0854  0.0870  0.0729  0.1043    0.0677   
2016-11-16 23:45:00                 0.0849  0.0868  0.0727  0.1042    0.0676   
2016-11-17 00:00:00                 0.0847  0.0867  0.0723  0.1038    0.0680   
2016-11-17 00:15:00                 0.0847  0.0864  0.0722  0.1034    0.0672   
2016-11-17 00:30:00                 0.0847  0.0864  0.0722  0.1033    0.0671   
2016-11-17 00:45:00                 0.0846  0.0864  0.0722  0.1032    0.0669   
2016-11-17 01:00:00                 0.0841  0.0862  0.0723  0.1032    0.0667   


##Continious update

In [ ]:
while True:
    try:
        Update() #get fresh data and send to graph
    except:
        retrys = 1
        print('Attempt to stream data failed')
        while retrys < 5: ##retry connecting 4 times
            time.sleep(160)  ##wait 2 min 
            try:
                Update() # have another go
            except:
                print('Retry' + str(retrys) + ' failed')
                retrys +=1 #increment try counter and try again
            else:
                retrys = 6 #set try counter so loop is exited
    time.sleep(14400)  ## update again in an hours time

##One off update

In [23]:
Update()